In [48]:
import pandas as pd
from openai import OpenAI
import json
client = OpenAI(api_key='')

In [4]:
meTooData = pd.read_csv('data/metooInINSTAGRAM_ME_TOO.csv')

In [126]:
meTooData['HASHTAGS'].iloc[0:1].values


array(['["#domesticviolence","#domesticabuse","#domesticviolenceawareness","#emotionalabuse","#abuse","#narcissisticabuse","#narcissist","#survivor","#gaslighting","#toxicrelationships","#mentalhealth","#ptsd","#trauma","#healing","#divorce","#womenempowerment","#narcissisticabusesurvivor","#women","#selflove","#sociopath","#love","#narcissism","#domesticviolencesurvivor","#manipulation","#toxicpeople","#abusiverelationship","#metoo","#abusesurvivor","#traumabonding","#psychologicalabuse"]'],
      dtype=object)

In [6]:
meTooData['DESCRIPTION'].head()

0    Freedom From Domestic Violence and Abuse is Wi...
1    .\n.\n.\n.\n.\n.\n.\n.\n.\n#feminist #feminism...
2    My aunt lived in that area, and we were taken ...
3    Die Termine und Themen unseres Gruppenangebote...
4    God I’m on whatever you on, do what you do ‼️ ...
Name: DESCRIPTION, dtype: object

In [19]:
def getEmbedding(text, model='text-embedding-3-small'):
    response = client.embeddings.create(input=text, model=model)
    return response

In [20]:
headData = meTooData['DESCRIPTION'].head()

embeddings = [getEmbedding(text) for text in headData]

In [23]:
print(embeddings)

[CreateEmbeddingResponse(data=[Embedding(embedding=[0.012585698626935482, -0.003786482848227024, 0.0564265139400959, 0.09531163424253464, -0.015957094728946686, -0.01357430312782526, -0.022573141381144524, 0.031686048954725266, 0.002205349039286375, 0.029683489352464676, 0.027934420853853226, 0.003412587335333228, 0.009943081997334957, 0.0158937219530344, 0.039316046983003616, 0.08152186870574951, 0.06261163204908371, 0.0270218625664711, -0.009898721240460873, 0.02386593073606491, -0.005792209412902594, 0.03820069879293442, -0.01097604725509882, -0.03898651525378227, -0.006793488748371601, -0.022687209770083427, 0.00541197694838047, -0.014524884521961212, 0.028492094948887825, -0.028187908232212067, 0.04905000329017639, -0.022268954664468765, -0.03221837431192398, 0.007623663172125816, 0.05485488846898079, 0.03181279078125954, 0.009670581668615341, -0.005693982820957899, -0.0008578997803851962, -0.015614884905517101, 0.01794697716832161, -0.053384654223918915, 0.012940581887960434, 0.0

In [42]:
MODEL = "gpt-3.5-turbo"
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an expert in sentiment analysis. You can ask me to analyze the sentiment of a text. I will provide a categorization of either positive, negative or neutral along with 2 key words that justify the sentiment."},
        {"role": "user", "content": "Can you tell me the sentiment of this text: I love this product!"},
    ],
    temperature=0.5,
)

In [106]:
def get_sentiment_analysis(text):
    response = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are an expert in sentiment analysis. You can ask me to analyze the sentiment of a text. I will provide a categorization of either positive, negative or neutral. The format of my response will be: Sentiment: {positive/negative/neutral}. I will not provide any further details."},
          {"role": "user", "content": "Can you tell me the sentiment of this text: {}".format(text)},
      ],
      temperature=0.5,
    )
    sentiment_json = json.dumps(json.loads(response.model_dump_json()), indent=4)
    sentiment_string = json.loads(sentiment_json)['choices'][0]['message']['content']
    return sentiment_string

In [107]:
# sentiment = get_sentiment_analysis("I love this product!")
sentiment_metoo = get_sentiment_analysis("I was sexually harassed at work today.")

In [108]:
# print(sentiment)
print(sentiment_metoo)

Sentiment: negative.


In [72]:
sentiment = [get_sentiment_analysis(text) for text in headData]

In [75]:
print([sentiment for sentiment in sentiment])

['Sentiment: Positive Keywords: Freedom, Reach', 'Sentiment: Positive Keywords: love, empowerment', 'Sentiment: Negative \nKeywords: sexually abuse, tied up', 'Sentiment: Neutral\nKeywords: Termine, Themen\n\nThe text provides information about the upcoming events and topics of the group offering "Ressourceninsel" in March. It includes details about the activities and how to register for them. The content is informative and lacks emotional tone, hence the neutral sentiment.', 'Sentiment: positive \nKeywords: pray, hope \n\n']


In [109]:

import re
def get_sentiment(sentiment):
    sentiment = sentiment.replace('\n', ' ')
    # remove punctuation
    sentiment = re.sub(r'[^\w\s]', '', sentiment)
    return sentiment.split(' ')[1].lower()

sentiment_keywords = [get_sentiment(sentiment) for sentiment in sentiment]

In [111]:
print(sentiment_keywords)

['positive', 'positive', 'negative', 'neutral', 'positive']


In [112]:
sentiment_metoo = get_sentiment(sentiment_metoo)

In [113]:
print(sentiment_metoo)

negative


In [118]:
# get rows where number of comments is greater than 1
len(meTooData[meTooData['COMMENTS_COUNT'] > 1])

141